In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve
import numpy as np
from scipy import stats
import joblib

def k():
    l = pd.ExcelFile('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx')
    m = []
    for n in l.sheet_names:
        o = pd.read_excel('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx', sheet_name=n)
        o['loc'] = n
        m.append(o)
    p = pd.concat(m)
    return p


def q(r):
    s = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    s = [t.lower() for t in list(s)]

    u = k()
    v = list(set(u['COMBO Plasma Box Number']))
    u = u[u['COMBO ID'].isin(r)]
    w = dict(zip(v, s))
    x = u['COMBO Plasma Box Number'].map(w)

    x.index = u['COMBO ID']
    u['COMBO Plasma Box Number'] = pd.Categorical(
        u['COMBO Plasma Box Number'])

    u['TB code'] = pd.Categorical(
        u['TB Classification'])

    u['code'] = u['COMBO Plasma Box Number'].cat.codes
    u['TB code'] = u['TB code'].cat.codes
    u.dropna(subset=['TB Classification'], inplace=True)
    return u, x


def y(z=True, aa=False):
    ab = pd.read_csv('data/protein_level_melted_merged.csv')
    ab['Pr_gn'] = ab['Protein.Group'] + '@' + ab['Genes']
    ab = pd.pivot_table(ab, columns='COMBO ID', index='Pr_gn', values='value')
    ac, ad = q(list(ab))
    ae = ac[ac['TB Classification'].isin(['Confirmed TB', 'Unlikely TB'])]
    af = ab[list(ae['COMBO ID'])].T
    if z:
        af = af.apply(lambda ag: stats.zscore(ag), axis=1)
    elif aa: 
        af = pd.DataFrame(preprocessing.minmax_scale(af, feature_range=(0, 1), axis=1, copy=True), index=af.index, columns=af.columns)
    ah = dict(zip(ae['COMBO ID'], ae['TB Classification']))
    ai = [ah[aj] for aj in list(af.index)]
    ai = [1 if ak == 'Confirmed TB' else 0 for ak in ai]
    return af, ai


def al(am=True, an=False):
    ao = pd.read_csv('data/protein_level_melted_merged.csv')
    ao['Pr_gn'] = ao['Protein.Group'] + '@' + ao['Genes']
    X = pd.pivot_table(ao, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = q(list(X))

    cls = meta[meta['TB Classification'].isin(['Unconfirmed TB'])]
    prot_cl = X[list(cls['COMBO ID'])].T
    if am:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif an: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    cls = dict(zip(cls['COMBO ID'], cls['TB Classification']))
    y = [cls[x] for x in list(prot_cl.index)]
    y = [1 if x == 'Confirmed TB' else 0 for x in y]
    return prot_cl, y


def get_all(std=True, nmr=False):
    X = pd.read_csv('data/protein_level_melted_merged.csv')
    X['Pr_gn'] = X['Protein.Group'] + '@' + X['Genes']
    X = pd.pivot_table(X, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = q(list(X))
    prot_cl = X[list(meta['COMBO ID'])]
    if std:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif nmr: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    cls = dict(zip(meta['COMBO ID'], meta['TB Classification']))
    prot_cl = pd.melt(prot_cl, ignore_index=False)
    prot_cl['tb'] = prot_cl['COMBO ID'].map(cls)
    return prot_cl.reset_index()

kk = pd.read_csv('new_biosignature/all_AUC_combined_2.csv')
kk['pr'] = [x.split(',') for x in kk['comb']]
kk['gn'] = [[item.split('@')[1] for item in sublist] for sublist in kk['pr']]
kk = kk.loc[kk.groupby('nfeat')['prec70sens'].transform(max) == kk['prec70sens']]
kk = kk.loc[kk.groupby('nfeat')['auc'].transform(max) == kk['auc']]
kk


AssertionError: 

In [ ]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, matthews_corrcoef, f1_score
from sklearn import metrics
from itertools import combinations

from sklearn.model_selection import StratifiedKFold


def parse_input():
    dd = pd.ExcelFile('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx')
    kk = []
    for x in dd.sheet_names:
        tmp = pd.read_excel('meta/COMBO PRO Banked Samples Aliquot Clinical Data.xlsx', sheet_name=x)
        tmp['loc'] = x
        kk.append(tmp)
    tot = pd.concat(kk)
    return tot


def preprocess_metadata(flt):
    # metadata
    cols = "#E64B35B2", "#4DBBD5B2", "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2", "#7E6148B2"
    cols = [x.lower() for x in list(cols)]

    meta = parse_input()
    dd = list(set(meta['COMBO Plasma Box Number']))
    meta = meta[meta['COMBO ID'].isin(flt)]
    enc = dict(zip(dd, cols))
    col_colors = meta['COMBO Plasma Box Number'].map(enc)

    col_colors.index = meta['COMBO ID']
    meta['COMBO Plasma Box Number'] = pd.Categorical(
        meta['COMBO Plasma Box Number'])

    meta['TB code'] = pd.Categorical(
        meta['TB Classification'])

    meta['code'] = meta['COMBO Plasma Box Number'].cat.codes
    meta['TB code'] = meta['TB code'].cat.codes
    meta.dropna(subset=['TB Classification'], inplace=True)
    return meta, col_colors


def get_prot_cl(std=True, nmr=False):
    X = pd.read_csv('data/protein_level_melted_merged.csv')
    X['Pr_gn'] = X['Protein.Group'] + '@' + X['Genes']
    X = pd.pivot_table(X, columns='COMBO ID', index='Pr_gn', values='value')
    meta, col_colors = preprocess_metadata(list(X))
    cls = meta[meta['TB Classification'].isin(['Confirmed TB', 'Unlikely TB'])]
    prot_cl = X[list(cls['COMBO ID'])].T
    if std:
        prot_cl = prot_cl.apply(lambda x: stats.zscore(x), axis=1)
    elif nmr: 
        prot_cl = pd.DataFrame(preprocessing.minmax_scale(prot_cl, feature_range=(0, 1), axis=1, copy=True), index=prot_cl.index, columns=prot_cl.columns)
    else:
        pass
    cls = dict(zip(cls['COMBO ID'], cls['TB Classification']))
    y = [cls[x] for x in list(prot_cl.index)]
    #print(len([x for x in y if x=='Confirmed TB']), len(y))
    y = [1 if x == 'Confirmed TB' else 0 for x in y]
    return prot_cl, y


def test_auc_manual(nams, save=False):
    prot_cl, y = get_prot_cl(True, False)
    prot_cl = prot_cl[nams]
    X_train, X_test, y_train, y_test = train_test_split(
        prot_cl, y, random_state=1, stratify=y, test_size=0.25)
    clf = LogisticRegression(penalty='l2')
    if len(nams)==1:
        X_train = X_train.values.reshape(-1,1)
        X_test = X_test.values.reshape(-1,1)
    clf.fit(X_train, y_train)
    if save==True:
        joblib.dump(clf, "new_biosignature/model/model{}.pkl".format(str(len(nams))))
    y_pred_test = clf.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
    fpr, tpr, thresholds = roc_curve(
        y_test, clf.predict_proba(X_test)[:, 1], pos_label=1)
    df = pd.DataFrame([fpr, tpr, thresholds]).T
    df.columns=['fpr', 'tpr', 'thresh']
    df['clf'] = '{} proteins (AUC = {})'.format(
        str(len(nams)), metrics.auc(fpr, tpr).round(3))
    return df
    

def evaluate_pred(nmdf):
    plt.figure(figsize=(3, 3))
    col = ['#7c4199', '#5e9632', "#00A087B2", "#3C5488B2", "#F39B7FB2", "#8491B4B2", "#91D1C2B2", "#DC0000B2",]
    dmp = []
    perf = []
    rocv = []
    prot_cl, y = get_prot_cl(True, False)

    for maxprot in range(5,11):
        tokeep = nmdf.head(maxprot).index
        nm = '{} proteins'.format(str(maxprot))
        cols = col[maxprot-5]
        X_train, X_test, y_train, y_test = train_test_split(
            prot_cl[tokeep], y, random_state=1, stratify=y, test_size=0.25)
        clf = LogisticRegression(penalty='l2')
        clf.fit(X_train, y_train)
        y_pred_test = clf.predict(X_test)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()
        accuracy = accuracy_score(y_test, y_pred_test)
        precision = precision_score(y_test, y_pred_test)
        recall = recall_score(y_test, y_pred_test)
        f1 = f1_score(y_test, y_pred_test)
        mcc = matthews_corrcoef(y_test, y_pred_test)
        fpr, tpr, thresholds = metrics.roc_curve(
            y_test, clf.predict_proba(X_test)[:, 1], pos_label=1)
        df = pd.DataFrame([fpr, tpr, thresholds]).T
        df.columns=['fpr', 'tpr', 'thresh']
        df['clf'] = '{} (AUC = {})'.format(
            nm, metrics.auc(fpr, tpr).round(3))
        rocv.append(df)
        plt.plot(fpr, tpr, c=cols, label='{} ROC (area = {})'.format(
            nm, metrics.auc(fpr, tpr).round(3)))
        # # Custom settings for the plot
        print(metrics.auc(fpr, tpr).round(3),  nm)
        perf.append([nm, accuracy, precision, recall,
                    f1, metrics.auc(fpr, tpr), mcc])
        plt.plot([0, 1], [0, 1], linestyle='dashed', c='black')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        
    
    #pd.concat(rocv).to_csv('../figures/fig5/data/roc_curve_data.csv')
    plt.xlabel('1-Specificity(False Positive Rate)', fontsize=9)
    plt.ylabel('Sensitivity(True Positive Rate)', fontsize=9)
    plt.legend(loc="lower right", fontsize=8)
    plt.tick_params(axis='both', which='major', labelsize=6)
    plt.tick_params(axis='both', which='minor', labelsize=6)
    plt.tick_params(axis='both', which='both', length=0)
    plt.axis('tight')
    plt.rcParams["axes.grid"] = False
    plt.plot(1-0.7, 0.9,'ro', markersize=1)
    plt.plot(1-0.7, 0.9,'ro', markersize=1)
    plt.show()
    plt.close()
  
  

def get_coef_lasso(prot_cl, y):
    reg = LassoCV(cv=StratifiedKFold(20), max_iter=10000, tol=0.0001, random_state=1).fit(prot_cl, y)
    # reg = LassoCV(cv=20, max_iter=10000, tol=0.001, random_state=1).fit(prot_cl, y)

    print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
    print("Best score using built-in LassoCV: %f" % reg.score(prot_cl, y))
    coef = pd.Series(reg.coef_, index=prot_cl.columns)
    metr = []
    imp_coef = coef.sort_values()
    imp_coef = imp_coef[imp_coef!=0]
    imp_coef = imp_coef.to_frame()
    imp_coef['abs'] = imp_coef[0].abs()
    imp_coef = imp_coef.sort_values(by='abs', ascending=False)
    imp_coef.to_csv('output/imp_coef_lasso.csv')
    imp_coef.drop(0, axis=1, inplace=True)
    imp_coef['rank_lasso'] = imp_coef['abs'].rank(ascending=False)
    return imp_coef

assert False
t = 50
nanprot = pd.read_csv('new_biosignature/percentage_missing_allproteins_TBClassfiication.csv')
nanprot = nanprot[nanprot['TB Classification'].isin(['Confirmed TB', 'Unlikely TB'])]
nanprot = nanprot.groupby('Genes').mean(['% miss']).reset_index()
nanprot = nanprot[nanprot['% miss']<=t]  
genes_to_keep = nanprot['Genes'].tolist()
prot_cl, y = get_prot_cl(True, False)
prot_cl = prot_cl[[col for col in prot_cl.columns if col.split('@')[-1] in genes_to_keep]]
lassocoef = get_coef_lasso(prot_cl,y)
coefs = lassocoef[lassocoef['abs']!=0]
coefs.to_csv('new_biosignature/imp_coef_lasso.csv')


AssertionError: 

In [ ]:
## add the 1,2

for rr in [1,2]:
    ll = pd.read_csv('output/allAUC_lasso_{}.csv'.format(str(rr)))
    ll = ll[ll['prec70sesn'] == ll['prec70sesn'].max()]
    ll = ll[ll['auc'] == ll['auc'].max()]
    ll.columns = list(kk)[:ll.shape[1]]
    ll['nfeat'] = rr
    ll['pr'] = [x.split(',') for x in ll['comb']]
    ll['gn'] = [[item.split('@')[1] for item in sublist] for sublist in ll['pr']]
    ll['keep']=True
    kk = pd.concat([kk,ll])
    
kk.to_csv('new_biosignature/biosignature_output.csv')

NameError: name 'pd' is not defined

In [ ]:



tt = []    
for _,i in kk.iterrows():
    tt.append(test_auc_manual(i['comb'].split(','), save=True))
    
tt = pd.concat(tt)
tt.to_csv('new_biosignature/20250423_ROC_CV.csv')
kk2 = kk[kk['nfeat']>2]
### individual features AUC
xx = [x for x in kk2['comb']]
k = []
for x in xx:
    k.extend(x.split(','))
tt = []    
for i in set(k):
    tmp = test_auc_manual([i], save=False)
    tmp['clf'] = [x.replace('1 proteins', i.split('@')[-1]) for x in tmp['clf']]
    tt.append(tmp)
tt = pd.concat(tt)
tt.to_csv('new_biosignature/individual_AUC.csv')
    


In [14]:
lassocoef = pd.read_csv('new_biosignature/imp_coef_lasso.csv')

topn = []
for i in range(1,7):
    rocdf = test_auc_manual(lassocoef.head(i)['Pr_gn'], save=False)
    rocdf['t'] = 'TopN'
    rocdf['n'] = i
    topn.append(rocdf)

topn = pd.concat(topn)
tt = pd.read_csv('new_biosignature/20250423_ROC_CV.csv')
tt['n'] = [x.split(' ')[0] for x in tt['clf']]
tt['t'] = 'BestN'
tot = pd.concat([topn, tt])
tot.to_csv('new_biosignature/topN_vs_bestN.csv')

NameError: name 'auc' is not defined

In [8]:
from statsmodels.stats.proportion import proportion_confint


prot_cl, y = get_prot_cl(True, False)
thresh = {}
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    X_train, X_test, y_train, y_true = train_test_split(
    prot_cl[clf.feature_names_in_], y, random_state=1, stratify=y, test_size=0.25)

    y_pred_proba = clf.predict_proba(X_test)[:,1]
   # Step 1: Find the threshold for 70% specificity
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    specificities = 1 - fpr
    # Find the threshold that gives approximately 70% specificity
    target_specificity = 0.7
    closest_idx = np.argmin(np.abs(specificities - target_specificity))
    threshold_for_70_specificity = thresholds[closest_idx]
    y_pred_proba = (y_pred_proba >= threshold_for_70_specificity).astype(int)
    thresh[i] = threshold_for_70_specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_proba >= threshold_for_70_specificity).ravel()
    
    sensitivity = tp / (tp + fn)
    sensitivity_ci = proportion_confint(count=tp, nobs=tp + fn, alpha=0.05, method='beta')

    # Specificity calculation and 95% CI
    specificity = tn / (tn + fp)
    specificity_ci = proportion_confint(count=tn, nobs=tn + fp, alpha=0.05, method='beta')

    print(sensitivity_ci, specificity_ci)

thresh

['Q9ULI3@HEG1' 'O95445@APOM' 'P02760-206-352@AMBP']
(0.7347115495257919, 0.9788828629702774) (0.5565659841604231, 0.8165979838178479)
['P23381@WARS1' 'P24821@TNC' 'P16070@CD44' 'O95445@APOM']
(0.6143334890033929, 0.9228644879989344) (0.5765225621350379, 0.8321422248034659)
['P24821@TNC' 'P16070@CD44' 'O95445@APOM' 'P01772@IGHV3-33' 'P08253@MMP2']
(0.7347115495257919, 0.9788828629702774) (0.5368387712284394, 0.8008133531683659)
['P24821@TNC' 'P16070@CD44' 'O95445@APOM' 'P01593-23-117@IGKV1D-33'
 'P08253@MMP2' 'P08637-17-254@FCGR3A']
(0.7347115495257919, 0.9788828629702774) (0.5765225621350379, 0.8321422248034659)


{3: 0.36437688053403366,
 4: 0.3587352003339288,
 5: 0.37014582140290947,
 6: 0.4005228540038571}

In [9]:
## plot biosignature across all TB classes
import pandas as pd
import numpy as np


def norm_to_unlikely(grp):
    sub = grp[grp['tb']=='Unlikely TB']
    grp['value_norm'] = grp['value'] - np.mean(sub['value'].values)
    return grp


rocv = pd.read_csv('new_biosignature/biosignature_output.csv')
rocv = rocv[rocv['nfeat'].isin([5,6])]
rocv = [x.split(',') for x in rocv['comb']]
i = set([x for xs in rocv for x in xs])
dd = get_all()
dd2 = dd[dd['Pr_gn'].isin(i)]
dd2 = dd2.groupby(['Pr_gn']).apply(norm_to_unlikely).reset_index(drop=True)
dd2['Pr_gn'] = [x.split('@')[-1] for x in dd2['Pr_gn']]
dd2= dd2.groupby(['Pr_gn', 'tb'])['value_norm'].describe()
dd2.to_csv('new_biosignature/biosignature_clinical_sites.csv')
dd2

/var/folders/6w/j26fwf3s1gxbgknbxwsd30xh0000gn/T/ipykernel_71093/162969272.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dd2 = dd2.groupby(['Pr_gn']).apply(norm_to_unlikely).reset_index(drop=True)


count          mean       std       min       25%  \
Pr_gn     tb                                                                  
APOM      Confirmed TB    120.0 -6.344289e-01  0.974851 -4.212340 -1.013629   
          Healthy          19.0 -2.677963e-01  0.474514 -1.905862 -0.385522   
          Latent TB         8.0 -3.565217e-01  0.459439 -1.066290 -0.579971   
          Unconfirmed TB  115.0 -1.175321e-01  0.862637 -2.139070 -0.691817   
          Unlikely TB     211.0 -1.683751e-17  1.063269 -3.974710 -0.616203   
CD44      Confirmed TB    120.0 -1.288712e-01  1.170305 -2.563115 -0.976373   
          Healthy          19.0 -5.619918e-01  0.955370 -2.057963 -1.685751   
          Latent TB         8.0 -1.212868e-01  1.221013 -1.629120 -1.358590   
          Unconfirmed TB  115.0 -1.156377e-01  0.921728 -2.520706 -0.717572   
          Unlikely TB     211.0 -2.525626e-17  0.926081 -1.999125 -0.609512   
FCGR3A    Confirmed TB    120.0  2.829002e-01  1.025938 -2.136551 -0.192556   
          Healthy          19.0  4.951669e-01  0.855443 -1.748353  0.298745   
          Latent TB         8.0  6.404992e-01  0.437478 -0.120240  0.413302   
          Unconfirmed TB  115.0  3.691310e-01  0.898453 -2.038541 -0.029571   
          Unlikely TB     211.0 -1.262813e-17  1.036198 -2.250770 -1.015842   
IGHV3-33  Confirmed TB    120.0  2.878383e-01  0.842116 -2.046304  0.220546   
          Healthy          19.0 -5.144126e-01  1.257078 -1.748064 -1.686331   
          Latent TB         8.0 -1.331718e+00  0.857870 -1.947337 -1.725897   
          Unconfirmed TB  115.0  1.120925e-01  0.977217 -2.007757 -0.178755   
          Unlikely TB     211.0  0.000000e+00  1.016737 -2.206873 -0.657327   
IGKV1D-33 Confirmed TB    120.0  2.924313e-01  1.029463 -2.320770  0.020695   
          Healthy          19.0 -8.401842e-02  0.948254 -2.014881 -0.048282   
          Latent TB         8.0 -1.162913e-01  0.883406 -1.515735 -0.450102   
          Unconfirmed TB  115.0  2.430453e-01  1.045863 -2.965468 -0.074148   
          Unlikely TB     211.0 -2.525626e-17  0.953651 -2.522142 -0.717295   
MMP2      Confirmed TB    120.0 -1.546991e-01  1.060983 -2.289883 -0.971410   
          Healthy          19.0 -4.871208e-01  1.051160 -2.527128 -0.994876   
          Latent TB         8.0  1.117087e-01  0.443943 -0.355518 -0.289878   
          Unconfirmed TB  115.0  1.668576e-01  0.977265 -1.956503 -0.731724   
          Unlikely TB     211.0  0.000000e+00  0.971853 -2.321486 -0.728975   
TNC       Confirmed TB    120.0  4.445829e-01  1.045675 -1.993145 -0.327348   
          Healthy          19.0 -7.527058e-01  0.880361 -1.894073 -1.289499   
          Latent TB         8.0 -4.846689e-02  1.043339 -1.199043 -0.844789   
          Unconfirmed TB  115.0  1.821752e-01  0.967496 -1.621606 -0.610869   
          Unlikely TB     211.0 -2.946563e-17  0.935529 -2.029019 -0.669556   

                               50%       75%       max  
Pr_gn     tb                                            
APOM      Confirmed TB   -0.665212 -0.021033  1.267987  
          Healthy        -0.186836 -0.065707  0.358553  
          Latent TB      -0.385722 -0.043741  0.325274  
          Unconfirmed TB -0.089472  0.508606  2.031301  
          Unlikely TB    -0.027740  0.624559  2.425247  
CD44      Confirmed TB   -0.074566  0.491198  4.783658  
          Healthy        -0.209812  0.076837  0.973843  
          Latent TB       0.313229  0.502353  1.753398  
          Unconfirmed TB  0.028533  0.459892  3.091152  
          Unlikely TB     0.137459  0.484053  2.926551  
FCGR3A    Confirmed TB    0.541374  0.928988  2.427650  
          Healthy         0.794325  0.948327  1.510014  
          Latent TB       0.634309  0.909232  1.211700  
          Unconfirmed TB  0.586431  0.943147  2.363561  
          Unlikely TB     0.289302  0.719170  2.902060  
IGHV3-33  Confirmed TB    0.558963  0.800594  1.406095  
          Healthy        -1.492857  0.743106  0.988144  
          Latent TB

In [5]:
import glob
import pandas as pd
import numpy as np
import joblib



prot_cl = get_all()
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification']=='Unconfirmed TB']
prot_cl = prot_cl[prot_cl['COMBO ID'].isin(dd['COMBO ID'])]

tt = []
preds = []
idss = pd.read_csv('new_biosignature/biosignature_output.csv')
idss = [x.split(',') for x in list(set(idss['comb']))]
idss = dict(zip([len(x) for x  in idss], idss))
prot_cl = pd.pivot_table(data=prot_cl, columns='Pr_gn', index='COMBO ID', values='value')
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    tmp = prot_cl[idss[i]]
    ypred = pd.DataFrame(clf.predict_proba(tmp))
    #ypred[0] = [0 if x<=thresh[i] else 1 for x in ypred[1]]
    ypred[0] = [0 if x<=0.5 else 1 for x in ypred[1]]
    ypred['sid'] = prot_cl.index
    ypred['cc'] = i
    ytot = ypred.groupby([0]).size().to_frame()
    ytot['cc'] = '{} feature LR model'.format(i)
    tt.append(ytot)
    preds.append(ypred)    


tt = pd.concat(tt)
ff = pd.concat(preds)
ff.columns=['Class', 'LR probability', 'COMBO ID', 'model feature number']
ff.to_csv('prediction/pred_model.csv')
tt.columns = ['nr', 'cc']
preds=pd.concat(preds)
tt.reset_index(inplace=True)
tt[0].replace({0:'Negative', 1:'Positive'}, inplace=True)
preds.to_csv('new_biosignature/individual_prediction.csv')
tt.to_csv('new_biosignature/pred_count.csv')
tt

['Q9ULI3@HEG1' 'O95445@APOM' 'P02760-206-352@AMBP']
['P23381@WARS1' 'P24821@TNC' 'P16070@CD44' 'O95445@APOM']
['P24821@TNC' 'P16070@CD44' 'O95445@APOM' 'P01772@IGHV3-33' 'P08253@MMP2']
['P24821@TNC' 'P16070@CD44' 'O95445@APOM' 'P01593-23-117@IGKV1D-33'
 'P08253@MMP2' 'P08637-17-254@FCGR3A']


/var/folders/6w/j26fwf3s1gxbgknbxwsd30xh0000gn/T/ipykernel_93621/2462652663.py:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tt[0].replace({0:'Negative', 1:'Positive'}, inplace=True)


,0,nr,cc
0,Negative,24,3 feature LR model
1,Positive,91,3 feature LR model
2,Negative,18,4 feature LR model
3,Positive,97,4 feature LR model
4,Negative,17,5 feature LR model
5,Positive,98,5 feature LR model
6,Negative,30,6 feature LR model
7,Positive,85,6 feature LR model


In [11]:
from upsetplot import from_contents, plot
from upsetplot import UpSet
import matplotlib.pyplot as plt



preds[0]= preds[0].replace({0:'Negative', 1:'Positive'})
pr_tmp = preds[preds[0]=='Positive']
kk = {}
for i,grp in pr_tmp.groupby('cc'):
    kk[str(i)]=list(grp['sid'])

toplot = from_contents(kk)


fig = plt.figure(figsize=(2.5, 4))
upset = UpSet(toplot, sort_by='cardinality', show_counts=True)  # disable the default bar chart

upset.plot(fig=fig)

plt.savefig('revision_figures/Fig5_B.pdf', bbox_inches='tight', dpi=800)
plt.savefig('revision_figures/Fig5_B.svg', bbox_inches='tight', dpi=800)
plt.close()

/opt/anaconda3/lib/python3.12/site-packages/upsetplot/data.py:385: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)
/opt/anaconda3/lib/python3.12/site-packages/upsetplot/plotting.py:795: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  styles["linewidth"].fillna(1, inplace=True)
/opt/an

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis

def theme_Publication():
    sns.set_style("ticks")
    sns.set_context("paper", font_scale=1.2)
    sns.set_palette("colorblind")
    
    plt.rcParams["font.family"] = "Helvetica"
    plt.rcParams["figure.figsize"] = [1.5, 1.5]
    plt.rcParams["axes.linewidth"] = 1
    plt.rcParams["axes.edgecolor"] = "black"
    plt.rcParams["legend.frameon"] = False
    plt.rcParams["legend.loc"] = "lower center"
    plt.rcParams["legend.framealpha"] = 1
    plt.rcParams["legend.fontsize"] = 9
    plt.rcParams["legend.title_fontsize"] = 9
    plt.rcParams["axes.grid"] = False
    plt.rcParams["figure.subplot.left"] = 0
    plt.rcParams["figure.subplot.right"] = 1
    plt.rcParams["figure.subplot.bottom"] = 0
    plt.rcParams["figure.subplot.top"] = 1

    return None

tot2 = get_all(False, False)
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification'].isin(['Unconfirmed TB', 'Confirmed TB'])]
tot2 = tot2[tot2['COMBO ID'].isin(dd['COMBO ID'])]
tmp = tot2[tot2['tb']=='Confirmed TB']
z = dict(zip(tmp['COMBO ID'], tmp['tb']))
tot2 = pd.pivot_table(data=tot2, columns='Pr_gn', index='COMBO ID', values='value')
tt = {}
nrmodels = len(set(preds[0]))
for i, grp in preds.groupby(['sid']):
    tt[i[0]] = str(grp[grp[0]=='Positive'].shape[0])
tt.update(z)
tot3 = tot2
Xarr = StandardScaler(with_mean=True).fit_transform(tot3)

clf = LinearDiscriminantAnalysis(solver='svd', n_components=2).fit(Xarr, [tt[i] for i in tot2.index])
print(clf.explained_variance_ratio_)
lda_df = clf.transform(Xarr)


X_reduced = pd.DataFrame(lda_df)
X_reduced['variable'] = tot3.index
X_reduced['class'] = X_reduced['variable'].map(tt)
X_reduced['class'] = X_reduced['class'].replace({'0':'Negative Unconfirmed', '1': 'Positive 1/4 models', '2': 'Positive 2/4 models', '3': 'Positive 3/4 models', '4': 'Positive all models'})
X_reduced['nrfreat'] = str(i)
X_reduced.to_csv('new_biosignature/lda_projection.csv')
lda_loadings = pd.DataFrame(clf.scalings_, index=list(tot3)).reset_index()
totfeat = []
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    print(clf.feature_names_in_)
    totfeat.extend(clf.feature_names_in_)
totfeat = set(totfeat)
lda_loadings['biosignature'] = lda_loadings['index'].isin(totfeat)
lda_loadings[0] = np.abs(lda_loadings[0])
lda_loadings['rank'] = lda_loadings[0].rank(ascending=False)
lda_loadings[['Protein', 'Gene']] = lda_loadings['index'].str.split('@', expand=True)
lda_loadings.drop(['index'], axis=1, inplace=True)
lda_loadings.to_csv('new_biosignature/lda_loadings.csv')


AssertionError: 

In [6]:
tot2 = get_all(False, False)
dd = pd.read_csv('../meta/combo_ms_names_test.csv')
dd = dd[dd['TB Classification'].isin(['Latent TB', 'Healthy', ])]
tot2 = tot2[tot2['COMBO ID'].isin(dd['COMBO ID'])]
tt = dict(zip(tot2['COMBO ID'], tot2['tb']))
tot2 = pd.pivot_table(data=tot2, columns='Pr_gn', index='COMBO ID', values='value')

test = []
for i in range(3,7):
    clf = joblib.load('new_biosignature/model/model{}.pkl'.format(str(i)))
    tot3 = tot2[clf.feature_names_in_]
    Xarr = StandardScaler(with_mean=True).fit_transform(tot3)
    pca_df = PCA(n_components=2, svd_solver='full').fit_transform(Xarr)
    X_reduced = pd.DataFrame(pca_df)
    X_reduced['variable'] = tot2.index
    X_reduced['class'] = X_reduced['variable'].map(tt)
    X_reduced['nrfreat'] = str(i)
    
    test.append(X_reduced)
   
test = pd.concat(test)
test.to_csv('new_biosignature/healthy_latent_pca.csv')
    